In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import geopandas as gpd
import scanpy as sc
from tifffile import imread, imwrite
from csbdeep.utils import normalize
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
from scipy import sparse
from matplotlib.colors import ListedColormap

# Configuration for inline plotting
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


2025-07-14 08:48:25.940173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 08:48:25.988158: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from shapely.geometry import Polygon
import geopandas as gpd

def plot_mask_and_save_images(title, gdf, img, cmap, output_name=None, bbox=None):
    if bbox is not None:
        # Crop the image to the bounding box
        cropped_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
    else:
        cropped_img = img

    # Setup figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # --- Subplot 1: Cropped H&E image only ---
    axes[0].imshow(cropped_img, cmap='gray', origin='lower')
    axes[0].set_title(title)
    # axes[0].axis('off')

    # --- Filter polygons within bbox ---
    if bbox is not None:
        bbox_polygon = Polygon([
            (bbox[0], bbox[1]), (bbox[2], bbox[1]),
            (bbox[2], bbox[3]), (bbox[0], bbox[3])
        ])
        # filtered_gdf = gdf[gdf.intersects(bbox_polygon)]

        filtered_gdf = gdf[gdf.intersects(bbox_polygon)].copy()

        # 🔧 Align polygon coordinates to cropped image origin
        filtered_gdf['geometry'] = filtered_gdf['geometry'].translate(
            xoff=-bbox[0], yoff=-bbox[1]
        )

    else:
        filtered_gdf = gdf

    # --- Subplot 2: Mask only ---
    filtered_gdf.plot(cmap=cmap, ax=axes[1])
    axes[1].set_title("Segmentation Mask")
    # axes[1].axis('off')

    # --- Subplot 3: H&E + Segmentation Overlay ---
    axes[2].imshow(cropped_img, origin='lower')
    
    # Plot filled polygons over image
    filtered_gdf.plot(cmap=cmap, ax=axes[2], alpha=0.1)
    filtered_gdf.boundary.plot(ax=axes[2], edgecolor='yellow', linewidth=1)

    # 🔧 Align axes so segmentation sits on the image
    axes[2].set_xlim(0, cropped_img.shape[1])
    axes[2].set_ylim(cropped_img.shape[0], 0)
    
    axes[2].set_title("H&E + Segmentation")
    axes[2].axis('off')


    
    # Save or show
    if output_name is not None:
        plt.savefig(output_name, bbox_inches='tight')
    else:
        plt.show()


# Loading data

In [3]:
# General image plotting functions
def plot_mask_and_save_image(title, gdf, img, cmap, output_name=None, bbox=None):
    if bbox is not None:
        # Crop the image to the bounding box
        cropped_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
    else:
        cropped_img = img

    # Plot options
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the cropped image
    axes[0].imshow(cropped_img, cmap='gray', origin='lower')
    axes[0].set_title(title)
    axes[0].axis('off')

    # Create filtering polygon
    if bbox is not None:
        bbox_polygon = Polygon([(bbox[0], bbox[1]), (bbox[2], bbox[1]), (bbox[2], bbox[3]), (bbox[0], bbox[3])])
        # Filter for polygons in the box
        intersects_bbox = gdf['geometry'].intersects(bbox_polygon)
        filtered_gdf = gdf[intersects_bbox]
    else:
        filtered_gdf=gdf

    # Plot the filtered polygons on the second axis
    filtered_gdf.plot(cmap=cmap, ax=axes[1])
    axes[1].axis('off')
    axes[1].legend(loc='upper left', bbox_to_anchor=(1.05, 1))


    # Save the plot if output_name is provided
    if output_name is not None:
        plt.savefig(output_name, bbox_inches='tight')  # Use bbox_inches='tight' to include the legend
    else:
        plt.show()

def plot_gene_and_save_image(title, gdf, gene, img, adata, bbox=None, output_name=None):

    if bbox is not None:
        # Crop the image to the bounding box
        cropped_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
    else:
        cropped_img = img

    # Plot options
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the cropped image
    axes[0].imshow(cropped_img, cmap='gray', origin='lower')
    axes[0].set_title(title)
    axes[0].axis('off')

    # Create filtering polygon
    if bbox is not None:
        bbox_polygon = Polygon([(bbox[0], bbox[1]), (bbox[2], bbox[1]), (bbox[2], bbox[3]), (bbox[0], bbox[3])])


    # Find a gene of interest and merge with the geodataframe
    gene_expression = adata[:, gene].to_df()
    gene_expression['id'] = gene_expression.index
    merged_gdf = gdf.merge(gene_expression, left_on='id', right_on='id')

    if bbox is not None:
        # Filter for polygons in the box
        intersects_bbox = merged_gdf['geometry'].intersects(bbox_polygon)
        filtered_gdf = merged_gdf[intersects_bbox]
    else:
        filtered_gdf = merged_gdf

    # Plot the filtered polygons on the second axis
    filtered_gdf.plot(column=gene, cmap='inferno', legend=True, ax=axes[1])
    axes[1].set_title(gene)
    axes[1].axis('off')
    axes[1].legend(loc='upper left', bbox_to_anchor=(1.05, 1))

    # Save the plot if output_name is provided
    if output_name is not None:
        plt.savefig(output_name, bbox_inches='tight')  # Use bbox_inches='tight' to include the legend
    else:
        plt.show()

def plot_clusters_and_save_image(title, gdf, img, adata, bbox=None, 
                                 color_by_obs=None, output_name=None, color_list=None):
    color_list=["#7f0000","#808000","#483d8b","#008000","#bc8f8f","#008b8b","#4682b4","#000080","#d2691e","#9acd32","#8fbc8f","#800080","#b03060","#ff4500","#ffa500","#ffff00","#00ff00","#8a2be2","#00ff7f","#dc143c","#00ffff","#0000ff","#ff00ff","#1e90ff","#f0e68c","#90ee90","#add8e6","#ff1493","#7b68ee","#ee82ee"]
    if bbox is not None:
        cropped_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
    else:
        cropped_img = img

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].imshow(cropped_img, cmap='gray', origin='lower')
    axes[0].set_title(title)
    axes[0].axis('off')

    if bbox is not None:
        bbox_polygon = Polygon([(bbox[0], bbox[1]), (bbox[2], bbox[1]), (bbox[2], bbox[3]), (bbox[0], bbox[3])])

    unique_values = adata.obs[color_by_obs].astype('category').cat.categories
    num_categories = len(unique_values)

    if color_list is not None and len(color_list) >= num_categories:
        custom_cmap = ListedColormap(color_list[:num_categories], name='custom_cmap')
    else:
        # Use default tab20 colors if color_list is insufficient
        tab20_colors = plt.cm.tab20.colors[:num_categories]
        custom_cmap = ListedColormap(tab20_colors, name='custom_tab20_cmap')

    merged_gdf = gdf.merge(adata.obs[color_by_obs].astype('category'), left_on='id', right_index=True)

    if bbox is not None:
        intersects_bbox = merged_gdf['geometry'].intersects(bbox_polygon)
        filtered_gdf = merged_gdf[intersects_bbox]
    else:
        filtered_gdf = merged_gdf

    # Plot the filtered polygons on the second axis
    plot = filtered_gdf.plot(column=color_by_obs, cmap=custom_cmap, ax=axes[1], legend=True)
    axes[1].set_title(color_by_obs)
    legend = axes[1].get_legend()
    legend.set_bbox_to_anchor((1.05, 1))
    axes[1].axis('off')

    # Move legend outside the plot
    plot.get_legend().set_bbox_to_anchor((1.25, 1))

    if output_name is not None:
        plt.savefig(output_name, bbox_inches='tight')
    else:
        plt.show()

# Plotting function for nuclei area distribution
def plot_nuclei_area(gdf,area_cut_off):
    fig, axs = plt.subplots(1, 2, figsize=(15, 4))
    # Plot the histograms
    axs[0].hist(gdf['area'], bins=50, edgecolor='black')
    axs[0].set_title('Nuclei Area')

    axs[1].hist(gdf[gdf['area'] < area_cut_off]['area'], bins=50, edgecolor='black')
    axs[1].set_title('Nuclei Area Filtered:'+str(area_cut_off))

    plt.tight_layout()
    plt.show()

# Total UMI distribution plotting function
def total_umi(adata_, cut_off):
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))

    # Box plot
    axs[0].boxplot(adata_.obs["total_counts"], vert=False, widths=0.7, patch_artist=True, boxprops=dict(facecolor='skyblue'))
    axs[0].set_title('Total Counts')

    # Box plot after filtering
    axs[1].boxplot(adata_.obs["total_counts"][adata_.obs["total_counts"] > cut_off], vert=False, widths=0.7, patch_artist=True, boxprops=dict(facecolor='skyblue'))
    axs[1].set_title('Total Counts > ' + str(cut_off))

    # Remove y-axis ticks and labels
    for ax in axs:
        ax.get_yaxis().set_visible(False)

    plt.tight_layout()
    plt.show()
    
# coexpression summary
def calculate_summary(adata, gene_names):
    """
    Calculate the summary of the number of rows with both genes' expression == 0,
    each of the genes' expression == 0, and none of the genes' expression == 0.
    
    Parameters:
    - adata: AnnData object
    - gene_names: list of two gene names (strings)
    
    Returns:
    - summary: dictionary containing the counts
    """
    gene1, gene2 = gene_names

    # Get the indices of the genes
    col1 = adata.var_names.get_loc(gene1)
    col2 = adata.var_names.get_loc(gene2)
    
    # Extract the expression data for the two genes
    expr_data = adata.X[:, [col1, col2]].toarray()
    
    both_zero = np.sum((expr_data[:, 0] == 0) & (expr_data[:, 1] == 0))
    gene1_zero = np.sum((expr_data[:, 0] == 0) & (expr_data[:, 1] != 0))
    gene2_zero = np.sum((expr_data[:, 0] != 0) & (expr_data[:, 1] == 0))
    none_zero = np.sum((expr_data[:, 0] != 0) & (expr_data[:, 1] != 0))
    
    summary = {
        'both_zero': both_zero,
        'gene1_zero': gene1_zero,
        'gene2_zero': gene2_zero,
        'none_zero': none_zero
    }
    
    return summary


In [8]:
! ls /home/mounim/rawdata/IMMUNEX/PJ2410310_250214/IMAGE/HE_nanozoomer_tif/

IMMUNEX001_Visium_HE_x40_z0.tif     IMMUNEX010_Visium_HE_20_x20_z0.tif
IMMUNEX002_Visium_HE_x40_z0.tif     IMMUNEX011_Visium_HE_x40_z0.tif
IMMUNEX003_Visium_HE_20_x20_z0.tif  IMMUNEX012_Visium_HE_x40_z0.tif
IMMUNEX004_Visium_HE_20_x20_z0.tif  IMMUNEX013_Visium_HE_x40_z0.tif
IMMUNEX005_Visium_HE_20_x20_z0.tif  IMMUNEX014_Visium_HE_x40_z0.tif
IMMUNEX006_Visium_HE_20_x20_z0.tif  IMMUNEX015_Visium_HE_x40_z0.tif
IMMUNEX007_Visium_HE_x40_z0.tif     IMMUNEX016_Visium_HE_x40_z0.tif
IMMUNEX008_Visium_HE_x40_z0.tif     IMMUNEX017_Visium_HE_x40_z0.tif
IMMUNEX009_Visium_HE_20_x20_z0.tif  IMMUNEX018_Visium_HE_x40_z0.tif


In [9]:
visium_path = '/home/mounim/rawdata/IMMUNEX/OUTPUT/Visium_NSCLC_IMMUNEX004/outs/'
tiff_path = "/home/mounim/rawdata/IMMUNEX/PJ2410310_250214/IMAGE/HE_nanozoomer_tif/IMMUNEX004_Visium_HE_20_x20_z0.tif"

In [10]:
filename = tiff_path
img = imread(filename)


In [6]:
# plt.figure(figsize=(10, 10))
# plt.imshow(img)
# # plt.axis('off')  # Uncomment to hide the axis
# plt.show()


# Stadist

In [11]:
print("Image shape:", img.shape)
print("Image dtype:", img.dtype)


Image shape: (43520, 48000, 3)
Image dtype: uint8


In [12]:
# Load model
model = StarDist2D.from_pretrained('2D_versatile_he')


Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


2025-07-14 08:49:43.680224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Before prediction, the H&E image must be normalized. We use the percentile normalization, which scales pixel values by the specified min and max percentiles. These values may need to be adjusted as needed for other images based on the nuclei predictions.

min_percentile = 5
max_percentile = 95
img = normalize(img, min_percentile, max_percentile)


# Prediction

In [14]:
# With the pretrained model and the normalized image we can start the nuclei prediction. This step may take a long time and it is the reason we need to use a GPU.
block_size = 4096
prob_threshes = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0001]
nms_thresh = 0.001
min_overlap = 128
context = 128

In [16]:
1-1

0

In [15]:
from tqdm import tqdm

for prob_thresh in tqdm(prob_threshes):    
    labels, polys = model.predict_instances_big(
        img, axes='YXC', 
        block_size=block_size,
        prob_thresh=prob_thresh, 
        nms_thresh=nms_thresh,
        min_overlap=min_overlap, 
        context=context,
        normalizer=None, n_tiles=(4,4,1))
    # export
    
    # Creating a list to store Polygon geometries
    geometries = []
    
    # Iterating through each nuclei in the 'polys' DataFrame
    for nuclei in range(len(polys['coord'])):
    
        # Extracting coordinates for the current nuclei and converting them to (y, x) format
        coords = [(y, x) for x, y in zip(polys['coord'][nuclei][0], polys['coord'][nuclei][1])]
    
        # Creating a Polygon geometry from the coordinates
        geometries.append(Polygon(coords))
    
    # Creating a GeoDataFrame using the Polygon geometries
    gdf = gpd.GeoDataFrame(geometry=geometries)
    gdf['id'] = [f"ID_{i+1}" for i, _ in enumerate(gdf.index)]
    
    gdf.to_pickle(f"segmentation_{prob_thresh}_{nms_thresh}_{min_overlap}.pkl")
    



  0%|                                                                                                                                                                                                                          | 0/6 [00:00<?, ?it/s]

effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)



 17%|██████████████████████████████████▎                                                                                                                                                                           | 1/6 [25:22<2:06:54, 1522.97s/it]

effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)



 33%|████████████████████████████████████████████████████████████████████▋                                                                                                                                         | 2/6 [53:13<1:47:18, 1609.53s/it]

...


 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                    | 4/6 [2:02:09<1:04:10, 1925.24s/it]

effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)



 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                  | 5/6 [2:46:19<36:26, 2186.81s/it]

effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [3:50:30<00:00, 2305.10s/it]


- block_size: The size of the blocks (or tiles) into which the large image is divided for processing. Each block has dimensions of 4096 pixels by 4096 pixels with 3 channels (RGB).


- min_overlap: Minimum overlap between adjacent blocks to ensure that objects that lie on the boundaries of blocks are properly segmented. Set to 128 pixels in both the x and y directions, and no overlap in the channel dimension.
Low value (e.g., 32–64): Stitching may miss objects at tile edges — border artifacts.
Higher value (e.g., 128): Better continuity, but increases compute cost and memory usage.


- context: Context size around each block considered during processing. It helps in providing additional information from the surrounding areas of a block. Set to 128 pixels around each block in both the x and y directions.
Low context: Poor segmentation near tile edges (objects cut in half).
High context: More accurate segmentation at borders, but more overlap = more compute.


- prob_thresh: Probability threshold to remove low-confidence object predictions.
Lower value (e.g., 0.01): More objects detected, including low-confidence ones (but more false positives).
Higher value (e.g., 0.2): Fewer objects, only confident nuclei are segmented (but more false negatives).


- nms_thresh: Overlap threshold to perform non-maximum suppression.
Lower value (0.001): Very strict — suppresses even slight overlaps → may undersegment.
Higher value (0.3–0.5): Allows more overlap → more duplicates, potential over-segmentation.

# ROI Visualisation

In [ ]:
prob_thresh = 0.001


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [ ]:
gdf = pd.read_pickle(f"segmentation_{prob_thresh}_{nms_thresh}_{min_overlap}.pkl")
gdf.head()

In [ ]:
xmin, xmax = gdf.geometry.bounds['minx'].min(), gdf.geometry.bounds['maxx'].max()
ymin, ymax = gdf.geometry.bounds['miny'].min(), gdf.geometry.bounds['maxy'].max()

# Optionally expand the range a bit
pad = 222
bbox = (50000, 50000, 50000+pad, 50000+pad)
print(bbox)
# Define a single color cmap
cmap=ListedColormap(['grey'])

output_name = '-'.join(['roi1-.png'])

# Create Plot
plot_mask_and_save_image(
    title="Region of Interest 1", 
    gdf=gdf,
    bbox=bbox, 
    cmap=cmap,
    img=img,
    output_name = output_name
)
plt.show()

In [ ]:
plot_mask_and_save_images(
    title="Region of Interest 2", 
    gdf=gdf,
    bbox=bbox, 
    cmap=cmap,
    img=img,
    output_name = 'plot_' + f'segmentation_{prob_thresh}_{nms_thresh}_{min_overlap}' + output_name 
)
plt.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np
import numpy as np
import plotly.graph_objects as go
plt.show()
def interactive_overlay(img, gdf, bbox=None):
    # Normalize image to uint8 if needed
    if img.dtype != np.uint8:
        if img.max() <= 1.0:
            img = (img * 255).astype(np.uint8)
        else:
            img = img.astype(np.uint8)

    # Handle grayscale images
    if img.ndim == 2:
        img = np.stack([img] * 3, axis=-1)

    # If bbox is provided, crop image and clip gdf
    if bbox is not None:
        img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        gdf = gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]

    # Create figure and add image
    fig = go.Figure()
    fig.add_trace(go.Image(z=img))

    # Overlay segmentation boundaries as yellow lines
    for i, row in gdf.iterrows():
        if row.geometry.geom_type == 'Polygon':
            x, y = row.geometry.exterior.xy
            # Transform coordinates to image space
            x = np.array(x) - (bbox[0] if bbox else 0)
            y = np.array(y) - (bbox[1] if bbox else 0)
            fig.add_trace(go.Scatter(
                x=x, y=y,
                mode='lines',
                line=dict(color='lime', width=1),
                hoverinfo='skip',
                showlegend=False
            ))

    # Layout tuning
    fig.update_layout(
        title="Interactive Overlay: H&E + Segmentation",
        xaxis=dict(showgrid=False, visible=False, scaleanchor='y'),
        yaxis=dict(showgrid=False, visible=False, autorange='reversed'),
        margin=dict(l=0, r=0, t=30, b=0),
        dragmode="pan"
    )

    return fig


fig = interactive_overlay(img=img, gdf=gdf, bbox = bbox)
fig.show()


# Load adata

In [ ]:
visium_path = '/home/mounim/rawdata/IMMUNEX/OUTPUT/Visium_NSCLC_IMMUNEX012/outs/'
dir_base = visium_path + '/binned_outputs/square_002um/'
raw_h5_file = dir_base+'filtered_feature_bc_matrix.h5'
adata = sc.read_10x_h5(raw_h5_file)
adata.var_names_make_unique()
adata

In [ ]:
# load Metadata: spatial coordinates of the barcodes


In [ ]:
tissue_position_file = dir_base+'spatial/'+'tissue_positions.parquet'
df_tissue_positions=pd.read_parquet(tissue_position_file)

#Set the index of the dataframe to the barcodes
df_tissue_positions = df_tissue_positions.set_index('barcode')
# Create an index in the dataframe to check joins
df_tissue_positions['index']=df_tissue_positions.index
# Adding the tissue positions to the meta data
adata.obs =  pd.merge(adata.obs, df_tissue_positions, left_index=True, right_index=True)


# adata Geoframe

In [ ]:
geometry = [Point(xy) for xy in zip(df_tissue_positions['pxl_col_in_fullres'], df_tissue_positions['pxl_row_in_fullres'])]
gdf_coordinates = gpd.GeoDataFrame(df_tissue_positions, geometry=geometry)


next, identify the barcodes belonging to cell nuclei. 
Remove overlapping nuclei to keep barcodes that are uniquely assigned and generate a new annData object.



# Spatial join

In [ ]:
# Perform a spatial join to check which coordinates are in a cell nucleus
result_spatial_join = gpd.sjoin(gdf_coordinates, gdf, how='left', predicate='within')
# Identify nuclei associated barcodes and find barcodes that are in more than one nucleus
result_spatial_join['is_within_polygon'] = ~result_spatial_join['index_right'].isna()
barcodes_in_overlaping_polygons = pd.unique(result_spatial_join[result_spatial_join.duplicated(subset=['index'])]['index'])
result_spatial_join['is_not_in_an_polygon_overlap'] = ~result_spatial_join['index'].isin(barcodes_in_overlaping_polygons)
# Remove barcodes in overlapping nuclei
barcodes_in_one_polygon = result_spatial_join[result_spatial_join['is_within_polygon'] & result_spatial_join['is_not_in_an_polygon_overlap']]
# The AnnData object is filtered to only contain the barcodes that are in non-overlapping polygon regions
filtered_obs_mask = adata.obs_names.isin(barcodes_in_one_polygon['index'])
filtered_adata = adata[filtered_obs_mask,:]
# Add the results of the point spatial join to the Anndata object
filtered_adata.obs =  pd.merge(filtered_adata.obs, barcodes_in_one_polygon[['index','geometry','id','is_within_polygon','is_not_in_an_polygon_overlap']], left_index=True, right_index=True)


# Bins aggregation

In [ ]:
# Group the data by unique nucleous IDs
groupby_object = filtered_adata.obs.groupby(['id'], observed=True)
# Extract the gene expression counts from the AnnData object
counts = filtered_adata.X
# Obtain the number of unique nuclei and the number of genes in the expression data
N_groups = groupby_object.ngroups
N_genes = counts.shape[1]
# Initialize a sparse matrix to store the summed gene counts for each nucleus
summed_counts = sparse.lil_matrix((N_groups, N_genes))
# Lists to store the IDs of polygons and the current row index
polygon_id = []
row = 0
# Iterate over each unique polygon to calculate the sum of gene counts.
for polygons, idx_ in groupby_object.indices.items():
    summed_counts[row] = counts[idx_].sum(0)
    row += 1
    polygon_id.append(polygons)
# Create and AnnData object from the summed count matrix
summed_counts = summed_counts.tocsr()
grouped_filtered_adata = anndata.AnnData(X=summed_counts,obs=pd.DataFrame(polygon_id,columns=['id'],index=polygon_id),var=filtered_adata.var)


# Adata preprocessing

In [ ]:
# Store the area of each nucleus in the GeoDataframe
gdf['area'] = gdf['geometry'].area

# Plot the nuclei area distribution before and after filtering
plot_nuclei_area(gdf=gdf,area_cut_off=500)

# Calculate quality control metrics for the original AnnData object
sc.pp.calculate_qc_metrics(grouped_filtered_adata, inplace=True)

# Plot total UMI distribution
total_umi(grouped_filtered_adata, 100)

# Create a mask based on the 'id' column for values present in 'gdf' with 'area' less than 500
mask_area = grouped_filtered_adata.obs['id'].isin(gdf[gdf['area'] < 500].id)

# Create a mask based on the 'total_counts' column for values greater than 100
mask_count = grouped_filtered_adata.obs['total_counts'] > 100

# Apply both masks to the original AnnData to create a new filtered AnnData object
count_area_filtered_adata = grouped_filtered_adata[mask_area & mask_count, :]

# Calculate quality control metrics for the filtered AnnData object
sc.pp.calculate_qc_metrics(count_area_filtered_adata, inplace=True)



# Clustering

In [ ]:
# Normalize total counts for each cell in the AnnData object
sc.pp.normalize_total(count_area_filtered_adata, inplace=True)

# Logarithmize the values in the AnnData object after normalization
sc.pp.log1p(count_area_filtered_adata)

# Identify highly variable genes in the dataset using the Seurat method
sc.pp.highly_variable_genes(count_area_filtered_adata, flavor="seurat", n_top_genes=2000)

# Perform Principal Component Analysis (PCA) on the AnnData object
sc.pp.pca(count_area_filtered_adata)

# Build a neighborhood graph based on PCA components
sc.pp.neighbors(count_area_filtered_adata)

# Perform Leiden clustering on the neighborhood graph and store the results in 'clusters' column
sc.tl.leiden(count_area_filtered_adata, resolution=0.35, key_added="clusters")


In [ ]:
plot_clusters_and_save_image(title="Region of interest 1", 
                             gdf=gdf, img=img, adata=count_area_filtered_adata, 
                             bbox=(12844,7700,13760,8664), color_by_obs='clusters')


In [ ]:
genes = ['Lyz1']
for gene in genes:
    plot_gene_and_save_image(title="Region of interest 1", 
                         gdf=gdf, gene=gene, img=img, 
                         adata=count_area_filtered_adata, 
                         bbox=(12844,7700,13760,8664))


# Cluster annotation

In [ ]:
# cd64: FCGR1. Intestinal macrophages markers https://pmc.ncbi.nlm.nih.gov/articles/PMC4451680/ 
markers = { 'Paneth': 'Lyz1', 'Goblet': 'Muc2', 'Plasma Cell': 'Jchain', 'Enterocyte': 'Fabp2', 'T-cells': [ 'Cd3' + l for l in 'de' ], 'Macrophages': [ 'Fcgr1', 'Adgre1', 'Cd68' ] }
p = sc.pl.dotplot(count_area_filtered_adata, markers, groupby='clusters', dendrogram=True, return_fig=True)
p.add_totals().style(dot_edge_color='black', dot_edge_lw=0.5).show()
